## <center> Filter特征筛选+随机森林建模+网格搜索调优

&emsp;&emsp;接下来首先尝试Filter特征筛选+随机森林建模+网格搜索调优的策略来训练第一组模型，当然，在Part I的结尾我们曾尝试随机森林建模，现在再次调用随机森林来建模，一方面是快速回顾此前的内容，同时由于接下来需要进行模型融合，因此我们需要手动保存一些模型训练的中间结果，因此需要改写Part I中的建模流程。

### 1.Filter特征筛选

- 特征筛选思路与方法

&emsp;&emsp;在特征总数较多且特征矩阵较为稀疏时，需要考虑在模型训练前进行特征筛选。我们知道，树模型及树模型的集成模型存在一定的特征筛选机制，即每棵树在进行训练的时候会优先选择能最大程度提升子集纯度的特征进行划分，但当特征太多时，尽管最终结果不一定会受到冗余（无用）特征影响，但模型效率会大幅降低，因此面对树模型及树模型的集成模型，我们仍然需要考虑在实际建模前进行特征筛选，优先带入有效特征进行建模。

&emsp;&emsp;而一般来说，特征筛选的方式主要有两类，其一是通过某些统计量对特征进行评估，在实际模型训练开始之前挑选出那些更加有效的特征并最终完成筛选，例如我们可以通过相关系数计算，判断特征和标签之间的相关关系，然后选取相关系数较大的特征带入进行建模，这种方法也被称为Filter方法；此外，我们也可以通过模型来筛选有效特征，例如随机森林模型可以输出特征重要性，我们可以先快速训练一个随机森林模型，然后根据输出的特征重要性，筛选更重要的特征带入后续超参数优化及交叉验证过程（需要知道的是，对于单独一个模型来说，是否带入冗余特征对单次训练来说影响不大，但由于超参数优化和交叉验证需要重复进行多轮训练，此时冗余特征的影响就会指数级上升），这样的特征筛选过程也被称为Wrapper过程。

&emsp;&emsp;在接下来的随机森林与LightGBM的模型训练过程中，我们将分别使用Filter方法和Wrapper方法进行特征筛选，但其实不同方法也是可以互换的，即我们也可以采用RF+Filter和LightGBM+Wrapper策略，同学们可以课后自行尝试这些组合。

- Filter相关系数特征筛选过程

&emsp;&emsp;接下来，我们尝试使用相关系数进行特征筛选。在目前处理好的数据中，特征总数高达1742，并且特征矩阵整体较为稀疏：

In [75]:
train = pd.read_csv('preprocess/train.csv')
test = pd.read_csv('preprocess/test.csv')

In [5]:
train.shape

(201917, 1742)

In [8]:
train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,authorized_flag&1&purchase_amount,authorized_flag&1&installments,city_id&19&purchase_amount,city_id&19&installments,...,category_4_var,category_4_skew,category_4_sum,city_id_nunique,merchant_category_id_nunique,merchant_id_nunique,state_id_nunique,subsector_id_nunique,card_id_size,card_id_count
0,67,C_ID_92a2005557,5,2,1,-0.820283,-170.641218,0.0,-1.422815,0.0,...,0.054623,-3.811953,261.0,9,46,118,3,21,283,283
1,62,C_ID_3d0044924f,4,1,0,0.392913,-213.239185,507.0,-4.782308,7.0,...,0.075036,-3.073118,327.0,9,58,148,3,24,356,356
2,57,C_ID_d639edf6cd,2,2,0,0.688056,-28.528749,0.0,-0.705405,0.0,...,0.065011,-3.548480,41.0,5,9,14,2,8,44,44
3,70,C_ID_186d6a6901,4,3,0,0.142495,-54.145736,89.0,-0.707839,1.0,...,0.023523,-6.361110,82.0,7,28,57,5,15,84,84
4,72,C_ID_cdbd2c0db2,1,3,0,-0.159749,-88.966702,179.0,0.000000,0.0,...,0.091496,-2.668681,151.0,7,37,103,7,19,169,169


In [11]:
# 计算稀疏性
1 - np.count_nonzero(train) / train.size

0.7739293811412331

因此，我们需要在实际建模之前进行特征筛选，以及排除过于稀疏的特征。考虑到标签是连续变量，此处可以直接使用皮尔逊相关系数来进行特征筛选，筛选过程如下：

In [12]:
# 提取特征名称
features = train.columns.tolist()
features.remove("card_id")
features.remove("target")
featureSelect = features[:]

# 计算相关系数
corr = []
for fea in featureSelect:
    corr.append(abs(train[[fea, 'target']].fillna(0).corr().values[0][1]))

# 取top300的特征进行建模，具体数量可选
se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
feature_select = ['card_id'] + se[:300].index.tolist()

# 输出结果
train_RF = train[feature_select + ['target']]
test_RF = test[feature_select]

最终生成的train_RF、test_RF就将是后续带入随机森林建模的数据。

In [15]:
train_RF.head(5)

,card_id,purchase_month_max_hist,purchase_month_mean_hist,purchase_month_max,purchase_month_mean,purchase_month_min_new,purchase_hour_section_nunique_new,purchase_month_mean_new,purchase_month_min,purchase_month_min_hist,...,city_id&151&installments,merchant_category_id&836&purchase_amount,merchant_category_id&80&installments,purchase_day_max_hist,most_recent_purchases_range_skew,merchant_category_id&683&installments,purchase_amount_nunique,category_3&0&installments,purchase_day_diff_nunique,target
0,C_ID_92a2005557,12,8.708861,15,9.526502,13.0,4.0,13.739130,5,5,...,0.0,0.0,1.0,1,0.390758,0.0,227,0.0,3,-0.820283
1,C_ID_3d0044924f,11,6.110368,14,7.078652,12.0,3.0,12.157895,0,0,...,0.0,0.0,48.0,1,0.898998,25.0,238,-2.0,3,0.392913
2,C_ID_d639edf6cd,12,4.190476,15,4.636364,13.0,2.0,14.000000,0,0,...,0.0,0.0,0.0,1,2.676919,0.0,42,0.0,3,0.688056
3,C_ID_186d6a6901,12,9.426230,15,10.547619,13.0,3.0,13.521739,8,8,...,0.0,0.0,12.0,1,0.062816,6.0,78,-3.0,3,0.142495
4,C_ID_cdbd2c0db2,12,11.255102,15,12.319527,13.0,4.0,13.788732,10,10,...,0.0,0.0,19.0,1,-0.457565,3.0,156,-1.0,3,-0.159749


In [13]:
train_RF.shape

(201917, 302)

当然，我们也可以将上述过程封装为一个函数并写入Elo.py模块（自定义模块），方便后续反复调用。

In [14]:
def feature_select_pearson(train, test):
    """
    利用pearson系数进行相关性特征选择
    :param train:训练集
    :param test:测试集
    :return:经过特征选择后的训练集与测试集
    """
    print('feature_select...')
    features = train.columns.tolist()
    features.remove("card_id")
    features.remove("target")
    featureSelect = features[:]

    # 去掉缺失值比例超过0.99的
    for fea in features:
        if train[fea].isnull().sum() / train.shape[0] >= 0.99:
            featureSelect.remove(fea)

    # 进行pearson相关性计算
    corr = []
    for fea in featureSelect:
        corr.append(abs(train[[fea, 'target']].fillna(0).corr().values[0][1]))

    # 取top300的特征进行建模，具体数量可选
    se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
    feature_select = ['card_id'] + se[:300].index.tolist()
    print('done')
    return train[feature_select + ['target']], test[feature_select]

### 2.随机森林模型训练与超参数调优

- 网格搜索方法介绍

&emsp;&emsp;接下来在挑选的特征中进行模型训练，当然为了确保模型本身泛化能力，一般模型训练过程都是和超参数调优过程同步进行的，如果我们是借助sklearn框架执行上述过程，则在流程的各环节都能得到极大的简化，一种最终基本的方案就是利用网格搜索进行超参数调优。当然在sklearn中网格搜索总共有三种，分别是网格搜索（GridSearchCV）、随机网格搜索（RandomGirdSearchCV）、对半网格搜索（HalvingGridSearchCV）以及随机对半网格搜索（RandomHalvingGridSearchCV），四种网格搜索尽管流程上有差异，但基本思路一致，都是通过不断的计算各组不同超参数组合输出的最终结果，并配合交叉验证过程，来寻找一组泛化能力最强的超参数组合。

&emsp;&emsp;当然，除了网格搜索以外，常用的超参数搜索方法还有TPE搜索、贝叶斯优化器搜索等，不同于网格搜索的暴力枚举过程，这些优化器能够借助贝叶斯过程进行一定程度的先验计算，并在实际搜索过程中不断的调整先验的判断，最终通过先验的判断提前剔除（或者选择）一部分组合，进而加快整体搜索过程。本次案例中我们将率先使用网格搜索，在后续的模型中将继续展示其他优化器的超参数搜索调优过程。不过需要知道的是，在sklearn框架内，模型和优化器接口较为统一，因此整体流程会更加简洁，但从计算效率角度考虑，贝叶斯优化器的搜索效率往往更高。

- 网格搜索基本流程

&emsp;&emsp;我们可以通过如下示例快速了解网格搜索在实际挑选超参数的计算流程：

<center><img src="https://s2.loli.net/2021/12/08/BAUqI5cf6uFavND.png" alt="image-20211208154708525" style="zoom:50%;" />

接下来我们通过定义一个完整的搜索函数来执行随机森林的网格搜索过程：

In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [26]:
def param_grid_search(train):
    """
    网格搜索参数调优
    :param train:训练集
    :return:网格搜索训练结果
    """
    # Step 1.创建网格搜索空间
    print('param_grid_search')
    features = train.columns.tolist()
    features.remove("card_id")
    features.remove("target")
    parameter_space = {
        "n_estimators": [81], 
        "min_samples_leaf": [31],
        "min_samples_split": [2],
        "max_depth": [10],
        "max_features": [80]
    }
    
    # Step 2.执行网格搜索过程
    print("Tuning hyper-parameters for mse")
    # 实例化随机森林模型
    clf = RandomForestRegressor(
        criterion="mse",
        n_jobs=15,
        random_state=22)
    # 带入网格搜索
    grid = GridSearchCV(clf, parameter_space, cv=2, scoring="neg_mean_squared_error")
    grid.fit(train[features].values, train['target'].values)
    
    # Step 3.输出网格搜索结果
    print("best_params_:")
    print(grid.best_params_)
    means = grid.cv_results_["mean_test_score"]
    stds = grid.cv_results_["std_test_score"]
    # 此处额外考虑观察交叉验证过程中不同超参数的
    for mean, std, params in zip(means, stds, grid.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return grid

In [27]:
grid = param_grid_search(train_RF)

param_grid_search
Tuning hyper-parameters for mse
best_params_:
{'max_depth': 10, 'max_features': 80, 'min_samples_leaf': 31, 'min_samples_split': 2, 'n_estimators': 81}
-13.617 (+/-0.088) for {'max_depth': 10, 'max_features': 80, 'min_samples_leaf': 31, 'min_samples_split': 2, 'n_estimators': 81}


In [30]:
grid

GridSearchCV(cv=2, estimator=RandomForestRegressor(n_jobs=15, random_state=22),
             param_grid={'max_depth': [10], 'max_features': [80],
                         'min_samples_leaf': [31], 'min_samples_split': [2],
                         'n_estimators': [81]},
             scoring='neg_mean_squared_error')

In [31]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, max_features=80, min_samples_leaf=31,
                      n_estimators=81, n_jobs=15, random_state=22)

至此，我们即完成了随机森林模型训练的相关工作，后续该函数也将写入Elo.py文件中。当然由于赛题要求评估指标为RMSE，因此我们可以通过如下方式计算模型最终在训练集上的RMSE计算结果。

In [33]:
np.sqrt(-grid.best_score_)

3.690154811274698

### 3.随机森林模型结果提交

&emsp;&emsp;当然，最终的建模成果还是要看模型在测试集上的表现，而竞赛中测试集标签是未知的，我们只有通过在线提交测试集结果的方式才能查看最终模型在测试集上的表现，因此我们需要通过如下方式对测试集数据进行预测，并按照结果提交形式写入本地文件：

In [50]:
test['target'] = grid.best_estimator_.predict(test[features])
test[['card_id', 'target']].to_csv("result/submission_randomforest.csv", index=False)

数据文档写入完毕后，接下来就可以直接在Kaggle上提交了。上传提交结果数据和下载数据过程类似，都可以直接利用网页功能实现，或者通过命令行的方式实现。

在Kaggle竞赛主页找到Late Submission进行结果提交，只需将结果文件在线提交即可：

<center><img src="https://i.loli.net/2021/10/23/ptkMKOXhv685LxG.png" alt="6a480d241a9a072ca921feedc9356c2" style="zoom:50%;" />

<center><img src="https://i.loli.net/2021/10/23/by9scX2gFER4jT7.png" alt="image-20211023183654754" style="zoom:50%;" />

&emsp;&emsp;提交完成后，即可在我的提交结果中看到成绩了：

<center><img src="https://i.loli.net/2021/10/23/KOqiElNHdPgGaV4.png" alt="image-20211023184031717" style="zoom:50%;" />

&emsp;&emsp;我们发现，随机森林建模结果约在前40%左右，至此，我们简单跑通了一次模型，并且顺利提交并看到排名，当然一般来说，在未进行优化之前的结果，我们会称其为Baseline，即模型效果基准线，后续我们将在此基础上进一步优化模型输出结果。

### 4.随机森林交叉验证评估与中间结果保存

&emsp;&emsp;在实际模型优化的过程中，有很多方法可以考虑，包括使用更加复杂高效的模型、进行模型融合、特征优化等等，但除此以外，还有一类经常被忽视但又同样高效优化的方法，那就是借助交叉验证进行多模型结果集成，当然此处所谓的多模型并不是采用了不同的评估器，而是同一个评估器（例如随机森林）在不同数据集上进行多次训练后生成多个模型，然后借助多个模型对测试集数据输出预测结果，最终通过取均值的方式来计算最终模型对测试集的预测结果。例如当前我们已经挑选了一组最优超参数，那么接下来就可以在这组超参数基础上进行五折交叉验证模型训练，该过程中对验证集的预测结果可以参与到后续Stacking融合过程中，而对测试集的预测结果则可以作为最终预测结果进行提交，相关过程如下所示：

<center><img src="https://s2.loli.net/2021/12/08/ALF3cfuSwmB7b8z.png" alt="image-20211208192640281" style="zoom:33%;" />

当然，交叉验证可以直接调用sklearn中评估器来实现，我们可以通过如下代码实现上述过程：

In [82]:
from sklearn.model_selection import KFold
from numpy.random import RandomState
from sklearn.metrics import mean_squared_error

In [79]:
def train_predict(train, test, best_clf):
    """
    进行训练和预测输出结果
    :param train:训练集
    :param test:测试集
    :param best_clf:最优的分类器模型
    :return:
    """
    
    # Step 1.选择特征
    print('train_predict...')
    features = train.columns.tolist()
    features.remove("card_id")
    features.remove("target")

    # Step 2.创建存储器
    # 测试集评分存储器
    prediction_test = 0
    # 交叉验证评分存储器
    cv_score = []
    # 验证集的预测结果
    prediction_train = pd.Series()
    
    # Step 3.交叉验证
    # 实例化交叉验证评估器
    kf = KFold(n_splits=5, random_state=22, shuffle=True)
    # 执行交叉验证过程
    for train_part_index, eval_index in kf.split(train[features], train['target']):
        # 在训练集上训练模型
        best_clf.fit(train[features].loc[train_part_index].values, train['target'].loc[train_part_index].values)
        # 模型训练完成后，输出测试集上预测结果并累加至prediction_test中
        prediction_test += best_clf.predict(test[features].values)
        # 输出验证集上预测结果，eval_pre为临时变量
        eval_pre = best_clf.predict(train[features].loc[eval_index].values)
        # 输出验证集上预测结果评分，评估指标为MSE
        score = np.sqrt(mean_squared_error(train['target'].loc[eval_index].values, eval_pre))
        # 将本轮验证集上的MSE计算结果添加至cv_score列表中
        cv_score.append(score)
        print(score)
        # 将验证集上的预测结果放到prediction_train中
        prediction_train = prediction_train.append(pd.Series(best_clf.predict(train[features].loc[eval_index]),
                                                             index=eval_index))
    
    # 打印每轮验证集得分、5轮验证集的平均得分
    print(cv_score, sum(cv_score) / 5)
    # 验证集上预测结果写入本地文件
    pd.Series(prediction_train.sort_index().values).to_csv("preprocess/train_randomforest.csv", index=False)
    # 测试集上平均得分写入本地文件
    pd.Series(prediction_test / 5).to_csv("preprocess/test_randomforest.csv", index=False)
    # 在测试集上加入target，也就是预测标签
    test['target'] = prediction_test / 5
    # 将测试集id和标签组成新的DataFrame并写入本地文件，该文件就是后续提交结果
    test[['card_id', 'target']].to_csv("result/submission_randomforest.csv", index=False)
    return

接下来尝试测试函数效果：

In [81]:
train_predict(train_RF, test_RF, grid.best_estimator_)

train_predict...


<ipython-input-79-3ec77b548cfd>:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  prediction_train = pd.Series()


3.675458048156077
3.7098960303168167
3.7175960057854875
3.682888749975916
3.646825949050688
[3.675458048156077, 3.7098960303168167, 3.7175960057854875, 3.682888749975916, 3.646825949050688] 3.686532956656997


此时，在本地文件中就能看到一个新的结果文件，该预测结果是交叉验证后各模型的预测结果的均值，相当于是一次简单的“集成”，我们可以继续在kaggle平台上提交该结果，查看测试集最终结果，能够发现，上述手动集成确实有效，前者是单模型结果，后者是手动集成后的模型结果：

<center><img src="https://s2.loli.net/2021/12/08/fvytIn9aMXrhEWC.png" alt="image-20211208193727983" style="zoom:50%;" />

<center><img src="https://s2.loli.net/2021/12/08/rY72cSFqjgPpThf.jpg" alt="111" style="zoom:50%;" />

而在后续建模过程中，我们还将频繁使用这种手动集成的方式来提高模型效果。